In [1]:
import chartify
import lime
import numpy as np
import pandas as pd
import pandas_profiling
import sklearn
import sklearn.ensemble
import sklearn.metrics
from lime import lime_text
from lime.lime_text import LimeTextExplainer
from sklearn import cluster
from sklearn.metrics import davies_bouldin_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import kneighbors_graph
from sklearn.preprocessing import StandardScaler

/Users/johannaanderson/anaconda/lib/python3.5/site-packages/sklearn/utils/fixes.py:313: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  _nan_object_mask = _nan_object_array != _nan_object_array


In [2]:
data = pd.read_csv('charity.csv')
data.head()

,ID,reg1,reg2,reg3,reg4,home,chld,hinc,genf,wrat,...,npro,tgif,lgif,rgif,tdon,tlag,agif,donr,damt,part
0,1,0,0,1,0,1,1,4,1,8,...,20,81,81,19,17,6,21.05,0.0,0.0,train
1,2,0,0,1,0,1,2,4,0,8,...,95,156,16,17,19,3,13.26,1.0,15.0,train
2,3,0,0,1,0,1,1,5,1,8,...,64,86,15,10,22,8,17.37,NaN,NaN,test
3,4,0,0,0,0,1,1,4,0,8,...,51,56,18,7,14,7,9.59,NaN,NaN,test
4,5,0,0,1,0,1,0,4,1,4,...,85,132,15,10,10,6,12.07,1.0,17.0,valid


In [3]:
np.random.seed(0)

In [4]:
pandas_profiling.ProfileReport(data)

Number of variables,24
Number of observations,8009
Total Missing (%),1.0%
Total size in memory,1.5 MiB
Average record size in memory,192.0 B
Numeric,16
Categorical,1
Boolean,6
Date,0
Text (Unique),0
Rejected,1


In [5]:
conditions = [
    data['reg1'] == 1,
    data['reg2'] == 1,
    data['reg3'] == 1,
    data['reg4'] == 1
]

outcomes = ['region1', 'region2', 'region3', 'region4']
test = np.select(conditions, outcomes, 'region5')
region = pd.Series(test).to_frame()
data['region'] = region
data.head(10)

,ID,reg1,reg2,reg3,reg4,home,chld,hinc,genf,wrat,...,tgif,lgif,rgif,tdon,tlag,agif,donr,damt,part,region
0,1,0,0,1,0,1,1,4,1,8,...,81,81,19,17,6,21.05,0.0,0.0,train,region3
1,2,0,0,1,0,1,2,4,0,8,...,156,16,17,19,3,13.26,1.0,15.0,train,region3
2,3,0,0,1,0,1,1,5,1,8,...,86,15,10,22,8,17.37,NaN,NaN,test,region3
3,4,0,0,0,0,1,1,4,0,8,...,56,18,7,14,7,9.59,NaN,NaN,test,region5
4,5,0,0,1,0,1,0,4,1,4,...,132,15,10,10,6,12.07,1.0,17.0,valid,region3
5,6,0,1,0,0,1,1,5,0,9,...,131,5,3,13,4,4.12,1.0,12.0,train,region2
6,7,0,0,0,0,1,3,4,0,8,...,74,6,5,22,3,6.50,0.0,0.0,valid,region5
7,8,0,0,0,0,1,3,2,0,5,...,41,4,2,20,7,3.45,0.0,0.0,valid,region5
8,9,0,0,1,0,1,2,3,1,5,...,160,28,34,14,4,14.00,NaN,NaN,test,region3
9,10,0,0,0,0,1,3,4,1,7,...,63,12,10,19,3,9.42,0.0,0.0,train,region5


In [14]:
notest = data.loc[data['part'] != 'test']
nopart = notest.drop(columns=['part', 'ID', 'damt'])
nopart.shape

(6002, 22)

In [23]:
byregprop = pd.crosstab(nopart.region, nopart.donr, normalize='index')
byregprop

donr,0.0,1.0
region,,
region1,0.458230,0.541770
region2,0.327412,0.672588
region3,0.644231,0.355769
region4,0.652830,0.347170
region5,0.660489,0.339511


In [25]:
byregcount = pd.crosstab(nopart.region, nopart.donr, margins=True)
#byregcount = byregcount.drop(index='All', axis=0)
byregcount.columns = ['non-donor', 'donor', 'total']
byregcount

,non-donor,donor,total
region,,,
region1,554,655,1209
region2,682,1401,2083
region3,469,259,728
region4,519,276,795
region5,784,403,1187
All,3008,2994,6002


In [26]:
byreg = (data.groupby('region')['donr'].sum().reset_index())
byreg

,region,donr
0,region1,655.0
1,region2,1401.0
2,region3,259.0
3,region4,276.0
4,region5,403.0


In [34]:
ch = chartify.Chart(blank_labels=True, x_axis_type='categorical')
ch.set_title("Population by region")
ch.plot.bar(data_frame=byreg,
           categorical_columns=['region'],
           numeric_column='donr')
ch.show('png')

TypeError: bar() got an unexpected keyword argument 'color'

In [ ]:
y = nopart.donr

In [ ]:
nopart.pop('donr')
nopart.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(nopart, y, test_size=0.3)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

In [ ]:
rf = sklearn.ensemble.RandomForestClassifier(n_estimators=500)
rf.fit(X_train, y_train)

In [ ]:
print(rf.feature_importances_)

In [ ]:
y_pred = rf.predict(X_test)
y_pred

In [ ]:
y_pp = rf.predict_proba(X_test)
y_pp

In [ ]:
print("Accuracy:", sklearn.metrics.accuracy_score(y_test, y_pred))

In [ ]:
sklearn.metrics.f1_score(y_test, y_pred, average='binary')

In [ ]:
pd.crosstab(y_test, y_pred)

In [ ]:
list(zip(X_train, rf.feature_importances_))

In [ ]:
from lime.lime_tabular import LimeTabularExplainer

In [ ]:
explainer = LimeTabularExplainer(
    X_train.values, feature_names = X_train.columns)

In [ ]:
exp = explainer.explain_instance(
    X_test.loc[64, :], rf[1].predict_proba, num_features=5
)

In [ ]:
exp.show_in_notebook(show_table=True)

In [ ]:
# Hierarchical clustering for unlabeled data
clus_data = StandardScaler().fit_transform(X_train)
# connectivity matrix for structured Ward
connectivity = kneighbors_graph(clus_data, n_neighbors=10, include_self=False)
# make connectivity symmetric
connectivity = 0.5 * (connectivity + connectivity.T)

ward = cluster.AgglomerativeClustering(n_clusters=5, linkage='ward', connectivity=connectivity)

In [ ]:
clust_mod = ward.fit(X_train)

In [ ]:
labels = clust_mod.labels_
label_df = pd.DataFrame(labels, columns=['Labels'])
label_df.shape

In [ ]:
clust_lab = X_train.copy()

In [ ]:
clust_lab['clust_label'] = labels

In [ ]:
clust_lab.head()

In [ ]:
with_id = pd.merge(clust_lab, data[['ID', 'donr']], how='inner', left_index=True, right_index=True)

In [ ]:
with_id.head()

In [ ]:
pd.crosstab(with_id.clust_label, with_id.donr)

In [ ]:
pd.crosstab(with_id.clust_label, with_id.donr).apply(lambda r: r/r.sum(), axis=1)

In [ ]:
# Evaluation metric for clustering algorithms where ground truth is not known.
# Lower is better, indicates better separation of the clusters. 
# n_clust = 3: 1.17
# n_clust = 4: 0.96
# n_clust = 5: 1.19

davies_bouldin_score(X_train, labels)